In [47]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [48]:
elo_hard=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)

elo_clay=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)


In [49]:
elo_data_hard =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19 --and Winner_Rank BETWEEN 1 AND 200 --and Winner_Rank<1000",
    con=devengine,)


elo_data_clay =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19",
    con=devengine,)

In [50]:
def get_player_record(player,opponent_rank,history,time,opponent,rank):
    player_history_top50=pd.DataFrame()
    player_history_top100=pd.DataFrame()
    player_history=pd.DataFrame()
    if opponent_rank<50:
        player_history_top50=history[(((history['Fav']==player)&(history['Dog_Rank']<50))|((history['Dog']==player)&(history['Fav_Rank']<50)))]
        #if player_history_top50.empty==False and (len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))  > 0.5:
            
            #print(time,player,"Top50",len(player_history_top50))
            #print(len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))        
    if opponent_rank<100:
        player_history_top100=history[(((history['Fav']==player)&(history['Dog_Rank']<100))|((history['Dog']==player)&(history['Fav_Rank']<100)))]
        #if player_history_top100.empty==False and (len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   > 0.5:     
        #    print(time,player,"Top100",len(player_history_top100))
        #    print(len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   
            
    else:
        opponent_rank_low=opponent_rank-50
        opponent_rank_high=opponent_rank+50
        player_history=history[((history['Fav']==player)&((history['Dog_Rank']>opponent_rank_low)&(history['Dog_Rank']<opponent_rank_high)))|
        ((history['Dog']==player)&((history['Fav_Rank']>opponent_rank_low)&(history['Fav_Rank']<opponent_rank_high)))]
        #if player_history.empty==False:
        #    if (len(player_history[player_history['Winner']==player])/len(player_history)) > 0.5 and len(player_history) >5:
        #        print(time,player,opponent_rank_low,opponent_rank_high,len(player_history))
        #        print(len(player_history[player_history['Winner']==player])/len(player_history))       
    
    return player_history,player_history_top100,player_history




In [51]:
def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


In [52]:
result_df=pd.DataFrame()
for _,row in elo_data_hard.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4:# and (((percent >= 0.5)&((percent2 < 0.5))) | ((percent2 >= 0.5)&((percent < 0.5)))):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav_Odds":[row.Fav_Odds],
                "Dog_Odds":[row.Dog_Odds],
                "Fav": [row.Fav],
                "Elo_Fav": [row.Elo_Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Odds":[row.Dog_Odds],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
                "percent": [percent],
                "percent2": [percent2],
                "Sex":[row.Sex]
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
#result_df      


In [53]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
if serving_womens.empty==False:
    serving=pd.concat([serving,serving_womens])
else:
    serving = serving
serving=serving.drop(columns='Time')
result=pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)
result_hard=result#[((result['percent']>0.5))|((result['percent2']>0.5))].copy()

In [54]:
result_df=pd.DataFrame()
for _,row in elo_data_clay.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4:# and (((percent >= 0.5)&((percent2 < 0.5))) | ((percent2 >= 0.5)&((percent < 0.5)))):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav_Odds":[row.Fav_Odds],
                "Dog_Odds":[row.Dog_Odds],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],"Sex":[row.Sex]
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
result_df      


""


In [55]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
serving=pd.concat([serving,serving_womens])
serving=serving.drop(columns='Time')
if result_df.empty==False:
    result=pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)
    result[(result['Fav_Serve%']>69)|(result['Dog_Serve%']>69)]
    clay=result
else:
    clay2=result_df
    clay2

In [56]:
elo_hard=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)
winner_dataset=elo_hard[['Date','Winner','Winner_Odds']].copy()
winner_dataset['Player']=winner_dataset['Winner']
winner_dataset['Odds']=winner_dataset['Winner_Odds']
winner_dataset['Win/Loss']='Win'
loser_dataset=elo_hard[['Date','Loser','Loser_Odds']].copy()
loser_dataset['Player']=loser_dataset['Loser']
loser_dataset['Odds']=loser_dataset['Loser_Odds']
loser_dataset['Win/Loss']='Loss'
winner_dataset.drop(columns=['Winner','Winner_Odds'],inplace=True)
loser_dataset.drop(columns=['Loser','Loser_Odds'],inplace=True)
data_concat=pd.concat([winner_dataset,loser_dataset])
data_concat=data_concat.sort_index()
data_concat['Odds']=data_concat.Odds.astype(float)

In [57]:
for _,matchup in result_hard.iterrows():
    player1=matchup.Fav
    player2=matchup.Dog
    player1_odds=float(matchup.Fav_Odds)
    player1_odds_hi=player1_odds+0.2
    player1_odds_lo=1
    player2_odds=float(matchup.Dog_Odds)
    player2_odds_hi=player2_odds+0.5
    player2_odds_lo=2
    player1=data_concat[data_concat['Player']==player1].copy()
    #player1['Streak'] = player1['Win/Loss'].groupby((player1['Win/Loss'] != player1['Win/Loss'].shift()).cumsum()).cumcount() + 1
    #player1['Streak']=player1.apply(lambda x: 'W'+str(x['Streak']) if x['Win/Loss']=='Win' else 'L'+str(x['Streak']),axis=1)
    #player1[player1['Odds']<1.4].tail(20)
    player2=data_concat[data_concat['Player']==player2].copy()
    #player2['Streak'] = player2['Win/Loss'].groupby((player2['Win/Loss'] != player2['Win/Loss'].shift()).cumsum()).cumcount() + 1
    #player2['Streak']=player2.apply(lambda x: 'W'+str(x['Streak']) if x['Win/Loss']=='Win' else 'L'+str(x['Streak']),axis=1)
    #player2[player2['Odds']>2.5].tail(20)
    player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
    if len(player2)>0:
        winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
    else:
        winperc2=0
    player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
    if len(player1)>0:
        winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
    else:
        winperc1=0
    if len(player1)>5 and len(player2)>3:
        print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
        #print(,matchup.Dog)
        print(len(player1),winperc1,len(player2),winperc2)

19:40 Xin Yu Wang (1-->1.59) Mirjam Bjorklund (2-->3.48)
14 0.6428571428571429 6 0.16666666666666666
21:00 Linda Fruhvirtova (1-->1.57) Tamara Zidansek (2-->3.56)
13 0.9230769230769231 9 0.1111111111111111
21:00 Rebeka Masarova (1-->1.85) Jasmine Paolini (2-->2.72)
14 0.7142857142857143 10 0.3
22:00 Adrian Andreev (1-->1.92) Daniel Masur (2-->2.54)
15 0.6 8 0.375
22:30 Anna Kalinskaya (1-->1.71) Lesia Tsurenko (2-->3.05)
18 0.7222222222222222 6 0.3333333333333333
22:30 Alison Van Uytvanck (1-->1.4) Tamara Korpatsch (2-->4.97)
10 0.8 8 0.5
23:30 Zhizhen Zhang (1-->1.47) Zdenek Kolar (2-->4.04)
14 0.6428571428571429 13 0.3076923076923077
23:30 Liam Broady (1-->1.39) Alexey Vatutin (2-->4.82)
14 0.9285714285714286 12 0.16666666666666666


In [58]:
#result_hard[result_hard['percent2']>0.5]
result_hard['Fav_Odds']=result_hard['Fav_Odds'].astype(float)
result_hard[((result_hard['Fav_Serve%']>75)|(result_hard['Dog_Serve%']>75))&(result_hard['percent2']<0.5)]
result_hard[(result_hard['percent2']>0.6)&(result_hard['percent']<0.4)&(result_hard['Fav_Odds']<1.8)]
#result_hard.to_clipboard(index=False)

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
1,21:00,1.39,2.86,Fa Rodriguez Taverna Santiago,Roberto Marcora,17%,6,Roberto Marcora,71%,7,0.166667,0.714286,Mens,NaN,NaN,NaN,NaN


In [59]:
#Dog
result_hard[(result_hard['percent2']>0)&(result_hard['percent2']>0.5)&(result_hard['percent']>0.2)&(result_hard['percent']<0.3) & (result_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [60]:
#Fav Pickem
result_hard[(result_hard['Fav_Odds']>1.8)& (result_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [61]:
#Fav Low Odds
result_hard[(result_hard['percent2']>0)&(result_hard['percent2']<0.5)&(result_hard['percent']>0.8)&(result_hard['Fav_Odds']>1.19)&(result_hard['Fav_Odds']<1.3)& (result_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [62]:
#Fav Low Odds
result_hard[(result_hard['percent2']<0.9)&(result_hard['percent2']>0.5)&(result_hard['percent']>0.8)&(result_hard['Fav_Odds']>1.19)&(result_hard['Fav_Odds']<1.3)& (result_hard['Sex']=='Mens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [63]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

In [64]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

In [65]:
#Women Low Odds Fav
result_hard[(result_hard['percent2']>0.2)&(result_hard['percent']<0.7)&(result_hard['Fav_Odds']>1.19)&(result_hard['Fav_Odds']<1.3)&(result_hard['Sex']=='Womens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [66]:
#Women Low Odds Fav
result_hard[(result_hard['percent2']<0.4)&(result_hard['percent']>0.6)&(result_hard['Fav_Odds']>1.19)&(result_hard['Fav_Odds']<1.3)&(result_hard['Sex']=='Womens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%


In [67]:
#Women Dog
result_hard[(result_hard['percent2']>0.5)&(result_hard['percent']<0.7)&(result_hard['Sex']=='Womens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
3,21:00,1.65,2.22,Rebeka Masarova,Rebeka Masarova,55%,11,Jasmine Paolini,55%,29,0.545455,0.551724,Womens,53.5,36.8,65.7,26.2


In [68]:
#Women Dog
result_hard[(result_hard['percent2']>0.6)&(result_hard['percent']<0.6)&(result_hard['Sex']=='Womens')]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,percent,percent2,Sex,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
